In [28]:
import random
type(random.uniform(-1, 1))

float

In [36]:
import numpy as np
import random
a = random.uniform(-1, 1)
b = random.uniform(-1, 1)
w = np.array([a, b])


In [37]:
w

array([-0.08261434,  0.69080893])

In [34]:
type(a)

float

In [33]:
b

-0.13150651809049374

In [3]:
import pygame

pygame 2.0.1 (SDL 2.0.14, Python 3.8.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq
%matplotlib inline

In [3]:
path = '1661-0.txt'
text = open(path, encoding='utf-8').read().lower()
# text is string datatype
print('corpus length:', len(text))

corpus length: 581888


In [4]:
text[:200]

"\ufeff\nproject gutenberg's the adventures of sherlock holmes, by arthur conan doyle\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, gi"

In [5]:
tokenizer = RegexpTokenizer('\w+')
words = tokenizer.tokenize(text)
words[:5]

['project', 'gutenberg', 's', 'the', 'adventures']

In [60]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))
# unique_word_index = {c: i for i, c in enumerate(unique_words)}
len(unique_word_index)

8201

In [63]:
unique_words[:5]

array(['0', '000', '1', '10', '100'], dtype='<U18')

In [7]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words)-WORD_LENGTH):
    prev_words.append(words[i: i+WORD_LENGTH])
    next_words.append(words[i+WORD_LENGTH])
print(prev_words[1])
print(next_words[1])

['gutenberg', 's', 'the', 'adventures', 'of']
sherlock


In [9]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    y[i, unique_word_index[next_words[i]]] = 1

In [10]:
X.shape

(109221, 5, 8201)

In [11]:
y.shape

(109221, 8201)

In [12]:
len(unique_words)

8201

In [13]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words), activation='softmax'))

In [14]:
optimizer = RMSprop(lr=0.01)
# since output is one_hot encoded we use c_c as loss and it also 2D array
# if output is 1D array we use s_c_c
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True)

Epoch 1/2
811/811 [==============================] - 79s 83ms/step - loss: 6.3658 - accuracy: 0.0827 - val_loss: 7.0421 - val_accuracy: 0.1018
Epoch 2/2
811/811 [==============================] - 59s 72ms/step - loss: 5.6386 - accuracy: 0.1458 - val_loss: 7.8937 - val_accuracy: 0.1064


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               4264960   
_________________________________________________________________
dense (Dense)                (None, 8201)              1057929   
Total params: 5,322,889
Trainable params: 5,322,889
Non-trainable params: 0
_________________________________________________________________


In [16]:
128*8201

1049728

In [17]:
1049728+8201

1057929

In [18]:
history.history

{'loss': [6.009746551513672, 5.767683982849121],
 'accuracy': [0.10848215222358704, 0.1473703533411026],
 'val_loss': [7.042056083679199, 7.893716812133789],
 'val_accuracy': [0.10179421305656433, 0.10637129098176956]}

In [54]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        x[0, t, unique_word_index[word]] = 1
    return x

In [65]:
def predict_words(text, n):
    X_test = prepare_input(text)
    y_pred = model.predict(X_test, verbose=0)[0]
    top_n_indices = heapq.nlargest(n, range(len(y_pred)), y_pred.take) # gives top 3 indices of max values
    return unique_words[top_n_indices]

In [74]:
predict_words("It is not a lack".lower(), 5)

array(['and', 'of', 'in', 'to', 'a'], dtype='<U18')

In [27]:
quotes = [
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages.",
    "That which does not kill us makes us stronger.",
    "I'm not upset that you lied to me, I'm upset that from now on I can't believe you.",
    "And those who were seen dancing were thought to be insane by those who could not hear the music.",
    "It is hard enough to remember my opinions, without also remembering my reasons for them!"
]

In [81]:
for q in quotes:
    l = q.split()[:5]
    s = ' '
    text = s.join(l)
    print(text) 
    # print(predict_words(text, 5))
    print()

It is not a lack

That which does not kill

I'm not upset that you

And those who were seen

It is hard enough to

